In [32]:
%gui osx
import py5
import numpy as np
import random
import copy
import time

In [33]:
def orientation(p, q, r): #megmondja a (p,q,r) irányítását
    
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
    if (val > 0):       #negatív irányítás
        return 1
    elif (val < 0):     #pozitív irányítás
        return 2
    else:              #kollineárisak
        return 0
  
def doIntersect(p1,q1,p2,q2): #megmondja, hogy a p1q1 és a p2q2 szakaszok metszenek-e
    
    if ((p1==p2) or (p1==q2) or (q1==p2) or (q1==q2)): #ha megegyeznek a végpontban, akkor nem metszőek
        return False 
    
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
  
    if ((o1 != o2) and (o3 != o4)):
        return True
    else:
        return False
   

In [34]:
class mPoint:
    def __init__(self,inx, iny, index=-1, szomszedok={}):
        self.x =inx
        self.y =iny
        self.index=index
        self.szomszedok=szomszedok
      
                   
    def __add__(self, other):
        return mPoint(self.x+other.x,self.y+other.y)
    
    def __sub__(self, other):
        return mPoint(self.x-other.x,self.y-other.y)
    
    def __str__(self):
        return str('('+str(self.index)+','+str(self.x)+','+str(self.y)+','+str(self.szomszedok)+')')
    
    def mag(self):
        return np.sqrt(self.magSq())
  
    def magSq(self):
        return self.x*self.x+self.y*self.y;
    
    def display(self):            #csak a pontot magát rajzolja ki
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
    
    def display_elestol(self):    #kirajzolja a pontot és a belőle kiinduló éleket is
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
        py5.stroke_weight(4)
        for csucs in self.szomszedok:
            py5.line(self.x,self.y,graf_csucsai.plist[csucs].x,graf_csucsai.plist[csucs].y)


class edge:
    global graf_elei
    def __init__(self, inStart,inEnd):
        self.s =inStart
        self.t =inEnd
        self.metszo=False
        
    def __str__(self):
        return '['+str(self.s.index)+','+str(self.t.index)+']'
       
    def display(self):
        py5.stroke_weight(4)
        if self.metszo==True:
            py5.stroke(204, 102, 0)
        else:
            py5.stroke(153)
            
        py5.line(self.s.x,self.s.y,self.t.x,self.t.y)
        
    def metsz_e_egy_konkretat(self,masikel):       #metszi-e a megadott élet
        return doIntersect(self.s,self.t,masikel.s,masikel.t)
   
        
    def metsz_e_valamit(self):                     #van-e olyan éle a gráfnak, amit ez metsz
        ideig=False
        for el in graf_elei.plist:
            if self.metsz_e_egy_konkretat(el):
                ideig=True
        self.metszo=ideig
        return ideig
   
class edge_set:
    
    def __init__(self, plist=[]):
        self.plist=plist
        
    def display(self):
        for x in self.plist:
            x.display()
            
    def __str__(self):
        szoveg=""
        for el in self.plist:
            szoveg+=str(el)
        return szoveg
    
    def van_e_metszes(self):      #a gráf élei között vannak-e metszőek
        ideig=False
        for x in self.plist:
            if x.metsz_e_valamit():
                ideig=True
        return ideig
    
    def el_benne_van_e(self, csucs1,csucs2): #az adott él szerepel-e a gráfban
        for el in self.plist:
            if ((el.s.index,el.t.index)==(csucs1,csucs2)) or ((el.s.index,el.t.index)==(csucs2,csucs1)):
                return True
        return False
        
        
        
class dinamic_pointset:
    def __init__(self, plist=[]):
        self.plist=plist
        self.active=False
        self.act_point=mPoint(0,0,0,[])
        self.log=False         
       
    def display(self):
        for x in self.plist:
            x.display()
    
    def select(self):
        if self.log:
            print('select')
        
        ref_point=mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)
        ref_point.display()
        dist_bound=30
        min_dist=dist_bound
        min_point=mPoint(0,0)
        for p in self.plist:
            if dist(ref_point,p)<min_dist:
                min_dist=dist(ref_point,p)
                min_point=p
        
        if min_dist<dist_bound:
            if self.log:
                print('succes select')
        
            self.active=True
            self.act_point=min_point
        else:
            if self.log:
                print('not succes select')
     #   self.plist.append(ref_point)
        
    def update(self):
        #print('update')
        if self.active:
            self.act_point.x=py5.mouse_x-py5.width/2
            self.act_point.y=py5.mouse_y-py5.height/2
    
    def relase(self):
        if self.log:
            print('re')
        
        self.active=False
   
    def do_something(self):
        if self.active:
            self.relase()
        else:
            self.select()
    
    def add_point(self,p):
        if not self.active:
            self.plist.append(p)
        else:
            if self.log:
                print('cant add now, point set is active')
 
def dist( a, b):
    return (a-b).mag();           
            
def mouse_point():
    return mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)

In [35]:
#ezt atirtam olyanra, hogy az input szerint csinalja

def random_adjacencia_matrix(num_of_points):
    matrix=np.random.randint(2,size=(num_of_points,num_of_points))
    for i in range(num_of_points):
        for j in range(num_of_points):
            if i==j:
                matrix[i][j]=0
            elif i<j:
                matrix[i][j]=matrix[j][i]
    return matrix

In [36]:
def create_graph(num_of_points):
    global graf_csucsai
    global graf_elei
    points=np.random.rand(num_of_points,2)*700-350
    adjacencia_matrix=random_adjacencia_matrix(num_of_points)
    ideig_ponthalmaz=[]
    i=0
        
    for x,y in points:
        random_csucshalmaz=set(j for j in range(num_of_points) if adjacencia_matrix[i][j]==1)
        ideig_ponthalmaz.append(mPoint(x,y,i,random_csucshalmaz))
        i+=1
    graf_csucsai=dinamic_pointset(ideig_ponthalmaz)
        
    ideiglenes_elhalmaz=[]
    for i in range(num_of_points):
        for j in range(num_of_points):
            if adjacencia_matrix[i][j]==1:
                ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[i],graf_csucsai.plist[j]))
            
    graf_elei=edge_set(ideiglenes_elhalmaz)

In [50]:
def create_graph2(num_of_points): #ez fog random gráfot generálni
    global graf_csucsai
    global graf_elei
    global eredeti_csucsok
    points=np.random.rand(num_of_points,2)*550-250
    ideig_ponthalmaz=[]
    i=0
        
    for x,y in points:
        ideig_ponthalmaz.append(mPoint(x,y,i,set({})))
        i+=1
    graf_csucsai=dinamic_pointset(ideig_ponthalmaz)
    eredeti_csucsok=copy.deepcopy(graf_csucsai)
        
    ideiglenes_elhalmaz=[]
    ideiglenes_elhalmaz_regi=[]
    for _ in range(num_of_points**nehezseg):  #itt jön be a nehézség, hogy hányszor próbál új élet behúzni
        csucs1=random.randint(0,num_of_points-1)
        csucs2=random.randint(0,num_of_points-1)
        graf_elei=edge_set(ideiglenes_elhalmaz)
        if (csucs1==csucs2) or graf_elei.el_benne_van_e(csucs1,csucs2): #olyan élet veszünk, ami még nincs a gráfban
            continue
        else:
            ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[csucs1],graf_csucsai.plist[csucs2]))
            graf_elei=edge_set(ideiglenes_elhalmaz)
            if graf_elei.van_e_metszes():   #ha van metszés az új éllel, akkor visszatérünk a régi listához
                ideiglenes_elhalmaz=ideiglenes_elhalmaz_regi.copy()
            else:                            #ha nincs metszés, akkor a régit frissítjük
                ideiglenes_elhalmaz_regi=ideiglenes_elhalmaz.copy()
            graf_elei=edge_set(ideiglenes_elhalmaz)

            
    for el in graf_elei.plist:   #frissítjük a csúcsoknál is a szomszédjaikat
        for i in range(len(graf_csucsai.plist)):
            if el.s.index==i:
                graf_csucsai.plist[i].szomszedok.add(el.t.index)
                eredeti_csucsok.plist[i].szomszedok.add(el.t.index)
            elif el.t.index==i:
                graf_csucsai.plist[i].szomszedok.add(el.s.index)  
                eredeti_csucsok.plist[i].szomszedok.add(el.s.index)
                
    shuffle_graph()

In [51]:
def shuffle_graph():
    new_points=np.random.rand(num_of_points,2)*550-250
    i=0
    for csucs in graf_csucsai.plist:
        csucs.x=new_points[i][0]
        csucs.y=new_points[i][1]
        i+=1
    

In [52]:
def setup():
    py5.size(1000, 700)
    py5.rect_mode(py5.CENTER)
    global photo
    photo=py5.load_image("youwin.jpeg")

def mouse_released(e):
    global graf_csucsai
    
    modifiers = e.get_modifiers()
    msgs = []
    if modifiers & e.SHIFT:
        graf_csucsai.add_point(mouse_point())
    else:
        graf_csucsai.do_something()

        

In [53]:
def masikdraw(): #teljesen ugyanaz, mint a másik, csak beszúr egy képet, ha kész vagy
    # adatok
    global graf_csucsai
    mmouse_x=py5.mouse_x-py5.width/2
    mmouse_y=py5.mouse_y-py5.height/2
    graf_csucsai.update()
    if not(graf_elei.van_e_metszes()):
        py5.image(photo, 0, 0)
    else:
        #stílus
        py5.background(255)
        py5.translate(py5.width/2, py5.height/2);
    
        #rajzolás
    
        py5.stroke(0)
        py5.stroke_weight(10)
        graf_csucsai.display()
        graf_elei.display()
    
        points=graf_csucsai.plist
        n=len(graf_csucsai.plist)
        points_raw=[(p.x,p.y) for p in graf_csucsai.plist]

In [54]:
global frame_counter
frame_counter=0
global arany
arany=0
max_frame=500

def nem_ez_a_draw(): #ez a sima draw
    # adatok
    global graf_csucsai
    global frame_counter
    global arany
    frame_counter=frame_counter+1
    
    if frame_counter<max_frame: #itt egyelore az van, hogy bizonyos ido utan elkezdenek majd maguktol a helyukre mozogni a csucsok, szoval ide kellene majd a gomb
        mmouse_x=py5.mouse_x-py5.width/2
        mmouse_y=py5.mouse_y-py5.height/2
        graf_csucsai.update()
        graf_elei.van_e_metszes()
    else:
        time.sleep(0.1)
        if arany<100:
            arany=arany+1
            for i in range(len(graf_csucsai.plist)): #itt mozognak a helyukre a csucsok
                graf_csucsai.plist[i].x=(1-(arany/100))*graf_csucsai.plist[i].x+(arany/100)*eredeti_csucsok.plist[i].x
                graf_csucsai.plist[i].y=(1-(arany/100))*graf_csucsai.plist[i].y+(arany/100)*eredeti_csucsok.plist[i].y
            graf_elei.van_e_metszes()
    
    #stílus
    py5.background(255)
    py5.translate(py5.width/2, py5.height/2);
    
    #rajzolás
    
    py5.stroke(0)
    py5.stroke_weight(10)
    graf_csucsai.display()
    graf_elei.display()
    
    points=graf_csucsai.plist
    n=len(graf_csucsai.plist)
    points_raw=[(p.x,p.y) for p in graf_csucsai.plist]
    

In [55]:
#EZ MOST AZ AKTUALIS
#van valami gomb, de akkor aktivalodik, ha raviszed az egeret, nem pedig, ha rakattintasz.. felsiker
#valahogy allitsuk at a grafot vagy a vasznat vagy valamit, hogy ne generaljon a gombhoz csucsokat
#a koordinatakat mindenkepp valahogy pontosabban kellene szamolni

global arany
arany = 0
global giveup_box_pressed
giveup_box_pressed = False

def draw(): #ez most a mindenes draw
    # adatok
    global graf_csucsai
    global arany
    global giveup_box_pressed #ez valahogy tenyleg pressed kellene legyen
    
    if py5.is_mouse_pressed:
        if -480 <= py5.mouse_x-py5.width/2 <= -280 and -340 <= py5.mouse_y-py5.height/2 <= -260: #itt allitodik at igazra, ha odaviszed az egeret
            giveup_box_pressed = True
    
    if not giveup_box_pressed: #ha nincs a gombon, akkor csak a szokasos
        mmouse_x=py5.mouse_x-py5.width/2
        mmouse_y=py5.mouse_y-py5.height/2
        graf_csucsai.update()
        graf_elei.van_e_metszes()
    else: #ha ott van a gombon, akkor helyremennek a csucsok
        time.sleep(0.05)
        if arany<100:
            arany=arany+1
            for i in range(len(graf_csucsai.plist)): #itt mozognak a helyukre a csucsok
                graf_csucsai.plist[i].x=(1-(arany/100))*graf_csucsai.plist[i].x+(arany/100)*eredeti_csucsok.plist[i].x
                graf_csucsai.plist[i].y=(1-(arany/100))*graf_csucsai.plist[i].y+(arany/100)*eredeti_csucsok.plist[i].y
            graf_elei.van_e_metszes()
    
    #stílus
    py5.background(255)
    py5.translate(py5.width/2, py5.height/2);
    
    #rajzolás
    #itt vannak a gomb cuccok
    py5.stroke(0) # Change the stroke color to black
    py5.fill(200) # Change the fill color to light gray
    py5.rect(-380, -300, 200, 80)
    
    py5.text_size(24)
    py5.fill(0) # Change the fill color to black
    py5.text_align(py5.CENTER, py5.CENTER)
    py5.text("Feladod?", -380, -300)
    
    py5.stroke(0)
    py5.stroke_weight(10)
    graf_csucsai.display()
    graf_elei.display()
    
    points=graf_csucsai.plist
    n=len(graf_csucsai.plist)
    points_raw=[(p.x,p.y) for p in graf_csucsai.plist]
    

In [56]:
#num_of_points=int(input('Mekkora gráfot szeretnél?'))
nehezseg=int(input('Milyen nehezet szeretnél (1, 2 vagy 3)?'))
num_of_points=10
create_graph2(num_of_points)
py5.run_sketch()

Milyen nehezet szeretnél (1, 2 vagy 3)?2


In [13]:
print(graf_elei)

[5,0][3,5][0,7][3,4][8,1][6,2][4,6][1,7][7,3][2,3][8,0][6,3][2,9][6,9][5,7][5,2][7,8][2,0][2,4]


In [14]:
for point in graf_csucsai.plist:
    print(point)

(0,290.3729748639453,126.57632264550818,{8, 2, 5, 7})
(1,217.75673905267388,88.92540138999067,{8, 7})
(2,-119.73158668710352,14.181085895909234,{0, 3, 4, 5, 6, 9})
(3,-153.7594317571733,259.94967245508883,{2, 4, 5, 6, 7})
(4,292.87117159605236,164.69121881418437,{2, 3, 6})
(5,156.71291546404484,-27.995302809112218,{0, 2, 3, 7})
(6,-104.00612887609037,228.71647805744823,{9, 2, 3, 4})
(7,-83.42641292794352,-243.19936280093106,{0, 1, 3, 5, 8})
(8,67.51690979685503,-212.93070740838346,{0, 1, 7})
(9,-141.77957595451892,-199.47106834294232,{2, 6})


In [15]:
for point in eredeti_csucsok.plist:
    print(point)

(0,191.12384781162524,282.7350999351661,{8, 2, 5, 7})
(1,234.7386174741177,207.9364623323201,{8, 7})
(2,22.785368986073422,297.5886457835833,{0, 3, 4, 5, 6, 9})
(3,-163.21328104960935,-196.19851931252586,{2, 4, 5, 6, 7})
(4,-97.03034211281161,-20.304186575266556,{2, 3, 6})
(5,-13.33395962069568,-23.47074135990698,{0, 2, 3, 7})
(6,-189.7693530440083,-122.51268462130867,{9, 2, 3, 4})
(7,148.43256325326058,92.4035108175326,{0, 1, 3, 5, 8})
(8,251.7303110097081,237.40083516443377,{0, 1, 7})
(9,-138.5509762569478,139.91158409738273,{2, 6})
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals